In [6]:
%matplotlib inline
%load_ext nb_black
import pandas as pd
import numpy as np
import random
import itertools

import matplotlib.pylab as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

from scipy.io import loadmat, savemat
from scipy.stats import f_oneway
from IPython.core.debugger import set_trace

from sklearn.feature_selection import mutual_info_regression

"""
monkey1
59 sessions
7 variables：(2380, 96)- spike count matrix, (2380,), (1, 2380), (2380,), (1, 2380)-orientation, (1, 1), (1, 2380)

monkey2
71 sessions

"""


"""sample to make the two monkey has comparable data size"""


def sample_sessions(monkey_num, save=False):
    data = loadmat("monkey" + "1" + ".mat")["monkey" + "1"][0]
    result = []
    for i in data:
        mask = i[0][0][-1][0] == max(i[0][0][-1][0])
        result.append([i[0][0][0][mask], i[0][0][-3][0][mask].reshape(1, -1)])
    samples = [
        filter_data_sample(result[session_num])[0].shape[0]
        for session_num in range(len(result))
    ]

    "sample from normal distribution"
    while True:
        norm_samples = np.random.normal(
            loc=np.mean(samples), scale=np.std(samples), size=len(get_data(monkey_num))
        ).round(0)
        if (norm_samples < 0).sum() == 0:
            norm_samples.sort()
            break

    "sampling"
    data = loadmat("monkey" + monkey_num + ".mat")["monkey" + monkey_num][0]
    result = []
    for i in data:
        mask = i[0][0][-1][0] == max(i[0][0][-1][0])
        result.append([i[0][0][0][mask], i[0][0][-3][0][mask].reshape(1, -1)])
    sample_s = pd.DataFrame(
        [
            filter_data_sample(result[session_num])[0].shape[0]
            for session_num in range(len(result))
        ],
        columns=["data2_samples"],
    )
    result = (
        sample_s.sort_values(by="data2_samples")
        .assign(ideal_samples=norm_samples)
        .apply(lambda x: min(x.values), 1)
    )
    return result.sort_index()


def sample_from_one_session(session_data, sample_number, save=False):
    sort_nums = np.arange(session_data[0].shape[0])
    random.shuffle(sort_nums)
    if save:
        return [
            session_data[0][sort_nums[: int(sample_number)]],
            session_data[1][sort_nums[: int(sample_number)]],
            session_data[2][:, sort_nums[: int(sample_number)]],
            session_data[3][sort_nums[: int(sample_number)]],
            session_data[4][:, sort_nums[: int(sample_number)]],
            session_data[5],
            session_data[6][:, sort_nums[: int(sample_number)]],
        ]
    else:
        return [
            session_data[0][sort_nums[: int(sample_number)]],
            session_data[1][:, sort_nums[: int(sample_number)]],
        ]


def get_data(monkey_index, bz=3, if_sample=False, filter_neurons=False):
    """
    i: seesion index, 0-counts_matrix, -3 orientation
    """
    data = loadmat("monkey" + monkey_index + ".mat")["monkey" + monkey_index][0]
    result = []
    for i in data:
        mask = i[0][0][-1][0] == max(i[0][0][-1][0])
        result.append([i[0][0][0][mask], i[0][0][-3][0][mask].reshape(1, -1)])
    if if_sample:
        sample_numbers = sample_sessions(monkey_index)
        result = [
            sample_from_one_session(result[i], sample_numbers[i])
            for i in range(len(sample_numbers))
        ]
    if filter_neurons:
        result_final = []
        for session_num in range(len(result)):
            data_sample = filter_data_sample(result[session_num])  # select one session
            pvalues = f_oneway(*list(split_data(data_sample, bin_size=bz).values()))[1]
            result_final.append([data_sample[0][:, pvalues < 0.05], data_sample[1]])
        result = result_final
    return result


def get_data_save(monkey_index, bz=3, if_sample=False, filter_neurons=False):
    """
    i: seesion index, 0-counts_matrix, -3 orientation
    """
    data = loadmat("monkey" + monkey_index + ".mat")["monkey" + monkey_index][0]
    result = []
    for i in data:
        mask = i[0][0][-1][0] == max(i[0][0][-1][0])
        result.append(
            [
                i[0][0][0][mask],
                i[0][0][1][mask],
                i[0][0][2][:, mask],
                i[0][0][3][mask],
                i[0][0][-3][:, mask],
                i[0][0][-2],
                i[0][0][-1][:, mask],
            ]
        )
    if if_sample:
        sample_numbers = sample_sessions(monkey_index, save=True)
        result = [
            sample_from_one_session(result[i], sample_numbers[i], save=True)
            for i in range(len(sample_numbers))
        ]
    if filter_neurons:
        result_final = []
        for session_num in range(len(result)):
            data_sample = filter_data_sample(
                result[session_num], save=True
            )  # select one session
            f_params = [
                i[0]
                for i in split_data(data_sample, bin_size=bz, save=True).values()
                if len(i[0]) > 0
            ]
            pvalues = f_oneway(*f_params)[1]
            result_final.append(
                [
                    data_sample[0][:, pvalues < 0.05],
                    data_sample[1],
                    data_sample[2],
                    data_sample[3],
                    data_sample[4],
                    data_sample[5],
                    data_sample[6],
                ]
            )
        result = result_final
    return result


def split_data(data_sample, bin_size, limit=0, save=False):
    #     "Counts_matrix",
    #                             "orientation",
    #                             "stimulus_class",
    #                             "trial_num",
    #                             "selected_class",
    #                             "session_number",
    #                             "contrast"
    if save:
        orientation = data_sample[-3].ravel()
    else:
        orientation = data_sample[1].ravel()
    groups = pd.cut(
        pd.Series(orientation),
        range(
            np.floor(orientation.min()).astype(int) - 1,
            np.ceil(orientation.max()).astype(int) + bin_size,
            bin_size,
        ),
    )
    df = pd.concat(
        [
            pd.DataFrame(data_sample[0]),
            pd.Series(orientation).rename("orientation"),
            groups.rename("group"),
        ],
        1,
    )
    if save:
        df = pd.concat(
            [
                df,
                pd.Series(data_sample[1]).rename("stimulus_class"),
                pd.Series(data_sample[2].ravel()).rename("trial_num"),
                pd.Series(data_sample[3]).rename("selected_class"),
                pd.Series(list(data_sample[5][0]) * len(data_sample[1])).rename(
                    "session_number"
                ),
                pd.Series(data_sample[6].ravel()).rename("contrast"),
            ],
            1,
        )
    return dict(
        df.groupby("group")
        .apply(
            lambda x: [x.drop("group", 1).values[:, :]]
            #             + [
            #                 each.ravel()
            #                 for each in np.split(x.drop("group", 1).values[:, -6:], range(1, 6), 1)
            #             ]
            #             if x.shape[0] >= limit
            #             else np.nan
        )
        .dropna()
    )


def bootstrap_sample(x, size_per_time, exp_num=1, times=100):
    sample = [
        (np.random.choice(x, size_per_time) ** exp_num).mean() for _ in range(times)
    ]
    return [np.mean(sample), np.std(sample)]


def plot_ten_neurons(dfs, select_p, exp_num=1, legend_on=True):
    x = [i.mid for i in list(dfs.keys()) if dfs[i].shape[0] > 0]
    for idx in select_p:
        y, yerr = zip(
            *[
                bootstrap_sample(i[:, idx], i.shape[0], exp_num)
                for i in list(dfs.values())
            ]
        )
        if list(dfs.values())[0].shape[1] < 100:
            if exp_num > 1:
                plt.plot(x, y)
                plt.fill_between(
                    x,
                    np.array(y) + np.array(yerr),
                    np.array(y) - np.array(yerr),
                    alpha=0.2,
                    #                     capsize=3,
                    label=r"$<r^" + str(exp_num) + "_{" + str(idx + 1) + "}|s>$",
                )
            else:
                plt.plot(
                    x, y, label=r"$<r^" + str(exp_num) + "_{" + str(idx + 1) + "}|s>$",
                )
                plt.fill_between(
                    x,
                    np.array(y) + np.array(yerr),
                    np.array(y) - np.array(yerr),
                    alpha=0.2,
                    #                     capsize=3,
                    #                     label=r"$<r_{" + str(idx + 1) + "}|s>$",
                )
        else:
            triu_index = np.triu_indices(96)
            plt.plot(x, y)
            plt.fill_between(
                x,
                np.array(y) + np.array(yerr),
                np.array(y) - np.array(yerr),
                alpha=0.2,
                #                     capsize=3,
                label=r"$<r_{"
                + str(triu_index[0][idx] + 1)
                + "} r_{"
                + str(triu_index[1][idx] + 1)
                + "}|s>$",
            )
    if exp_num == 1 and legend_on == True:
        plt.legend(frameon=False, ncol=2)
    else:
        plt.legend().remove()
    plt.xlabel("orientation")


#     plt.ylabel("average stimulus")


def normalize_across_trial(m, get_first=False):
    if get_first:
        return m[0] - m[0].mean(0)
    else:
        return m - m.mean(0)


def eigsorted(cov):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    return vals[order], vecs[:, order]


def draw_ellipse(x, y, ax, label, color):
    nstd = 2
    cov = np.cov(x, y)
    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    w, h = 2 * nstd * np.sqrt(vals)
    sc = plt.scatter(x, y, label=label, s=3, c=color)
    ell = Ellipse(
        xy=(np.mean(x), np.mean(y)),
        width=w,
        height=h,
        angle=theta,
        #         color=sc.get_facecolors()[0].tolist(),
        color=color,
        linestyle="-",
    )

    ell.set_facecolor("none")
    xmax, ymax = ell.get_patch_transform().transform(ell.get_path().vertices).max(0)
    xmin, ymin = ell.get_patch_transform().transform(ell.get_path().vertices).min(0)
    ax.add_artist(ell)
    ax.axes.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax.axes.yaxis.set_major_locator(plt.MaxNLocator(4))
    return xmin, ymin, xmax, ymax


def filter_data_sample(data_sample, save=False):
    if save:
        mask = np.array(
            [True if i >= 210 and i <= 330 else False for i in data_sample[-3][0]],
            dtype=bool,
        ) & np.array((data_sample[0] > 200).sum(1) == 0, dtype=bool)
        data_sample[0] = (data_sample[0][mask],)
        data_sample[1] = (data_sample[1][mask],)
        data_sample[2] = (data_sample[2][:, mask],)
        data_sample[3] = (data_sample[3][mask],)
        data_sample[-3] = (data_sample[-3][:, mask],)
        data_sample[-2] = (data_sample[-2],)
        data_sample[-1] = (data_sample[-1][:, mask],)
    else:
        mask = np.array(
            [True if i >= 210 and i <= 330 else False for i in data_sample[1][0]],
            dtype=bool,
        ) & np.array((data_sample[0] > 200).sum(1) == 0, dtype=bool)
        data_sample[0] = data_sample[0][mask, :]
        data_sample[1] = data_sample[1][:, mask]
    return data_sample


def get_index_list(pair_neuron_list):
    l = []
    for i in pair_neuron_list:
        ref_df = pd.DataFrame(np.concatenate(np.triu_indices(96)).reshape(2, -1)).T
        l.append(ref_df[(ref_df[0] == i[0]) & (ref_df[1] == i[1])].index[0])
    return l


def sensivity(x, y):
    return (x.mean(0) - y.mean(0)) / np.where(
        ((np.var(x, 0) + np.var(y, 0)) / 2) ** 0.5 == 0,
        np.nan,
        ((np.var(x, 0) + np.var(y, 0)) / 2) ** 0.5,
    )


"""
calculate sensitivity
"""


def all_sensitivity(data, savename, monkey_num, bz=3, cate="linear"):
    sens_total = []
    for d in data:
        data_sample = filter_data_sample(d)
        dfs = split_data(data_sample, bin_size=bz)
        if cate in ["cross", "square"]:
            norm_dfs = [normalize_across_trial(i) for i in list(dfs.values())]
            r_ij = np.expand_dims(np.concatenate(norm_dfs), 2) * np.expand_dims(
                np.concatenate(norm_dfs), 1
            )
            if cate == "cross":
                sep_index = np.triu_indices(96, k=1)
            else:
                sep_index = np.diag_indices(96)
            r_ij_flat = np.vstack([r_ij[i][sep_index] for i in range(r_ij.shape[0])])
            data_sample = filter_data_sample([r_ij_flat, data_sample[1]])
            dfs = split_data(data_sample, bin_size=bz)
        value_list = list(dfs.values())

        sens_total.extend(
            np.concatenate(
                [
                    sensivity(value_list[i + 1], value_list[i])
                    for i in range(len(dfs) - 1)
                    if value_list[i].shape[0] >= 10 and value_list[i + 1].shape[0] >= 10
                ]
            )
        )

    pd.Series(sens_total).hist(bins=np.linspace(-2, 2, 100))

    plt.title("monkey " + monkey_num + " all sessions, " + str(cate))
    plt.grid(False)
    plt.savefig(savename + "_filter.pdf")


def single_sensitivity(data, session_num, savename, bz=3, cate="linear", save=True):
    sens_total = []
    data_sample = filter_data_sample(data[session_num])
    dfs = split_data(data_sample, bin_size=bz)
    if cate in ["cross", "square"]:
        norm_dfs = [normalize_across_trial(i) for i in list(dfs.values())]
        r_ij = np.expand_dims(np.concatenate(norm_dfs), 2) * np.expand_dims(
            np.concatenate(norm_dfs), 1
        )
        if cate == "cross":
            sep_index = np.triu_indices(96, k=1)
        else:
            sep_index = np.diag_indices(96)
        r_ij_flat = np.vstack([r_ij[i][sep_index] for i in range(r_ij.shape[0])])
        data_sample = filter_data_sample([r_ij_flat, data_sample[1]])
        dfs = split_data(data_sample, bin_size=bz)
    value_list = list(dfs.values())
    sens_total.extend(
        [
            [
                sensivity(value_list[i + 1], value_list[i]),
                list(dfs.keys())[i].mid,
                list(dfs.keys())[i + 1].mid,
            ]
            for i in range(len(dfs) - 1)
            if value_list[i].shape[0] >= 10 and value_list[i + 1].shape[0] >= 10
        ]
    )
    edge = int((len(sens_total) - 4) // 2)
    for s in sens_total[edge : edge + 4]:
        pd.Series(s[0]).hist(
            bins=np.linspace(-1, 1, 50),
            alpha=0.7,
            label="orientation " + str(s[1]) + " " + str(s[2]),
        )
    plt.legend()
    plt.title(
        "monkey "
        + str(monkey_num)
        + ", session: "
        + str(session_num)
        + ", bin size: "
        + str(bz)
        + ", "
        + str(cate)
    )
    plt.grid(False)
    if save:
        plt.savefig(savename + "_filter.pdf")


def change_dtypes(aa):
    revise_dtypes = {
        "Counts_matrix": "uint8",
        "orientation": "double",
        "stimulus_class": "<U1",
        "trial_num": "uint16",
        "selected_class": "<U1",
        "session_number": "int",
        "contrast": "double",
    }
    for key, item in aa.items():
        if key == "session_number":
            aa[key] = item[0]
        elif key in ["trial_num", "contrast", "orientation"]:
            aa[key] = item.reshape(1, -1).astype(revise_dtypes[key])
        else:
            aa[key] = item.astype(revise_dtypes[key])
    return aa


params = {
    "legend.fontsize": 14,
    "legend.frameon": False,
    "ytick.labelsize": 14,
    "xtick.labelsize": 14,
    "figure.dpi": 300,
    "axes.labelsize": 14,
    "axes.titlesize": 14,
    "pdf.fonttype": 42,
    "font.sans-serif": "Myriad Pro",
    "font.family": "sans-serif",
}
plt.rcParams.update(params)


monkey_num = "2"
bz = 3  # bin size is 3
session_num = 20

<IPython.core.display.Javascript object>

In [ ]:
sample_limit = 5
filter_neurons = False
data = get_data(monkey_num, if_sample=False, filter_neurons=filter_neurons)
data_sample = filter_data_sample(data[session_num])  # select one session
dfs = split_data(data_sample, bin_size=bz)  # split stimulus data (r) based on bin size
dfs = {k: dfs[k][0] for k in dfs.keys()}
norm_dfs = [
    normalize_across_trial(i)
    for i in list(dfs.values())
    if normalize_across_trial(i).shape[0] >= sample_limit
]
unnorm_dfs = [i for i in list(dfs.values()) if i.shape[0] >= sample_limit]

key_list = [k for k in list(dfs.keys()) if dfs[k].shape[0] >= sample_limit]

### Row 1 A

In [ ]:
for monkey in ["1", "2"]:
    pd.Series([filter_data_sample(i)[0].shape[0] for i in get_data(monkey)]).hist(
            grid=False, bins=range(0, 2000, 100), alpha=0.7
        )
    print(
        np.mean([filter_data_sample(i)[0].shape[0] for i in get_data(monkey)]),
        np.std([filter_data_sample(i)[0].shape[0] for i in get_data(monkey)]),
    )
plt.xlabel("number of trials")
plt.ylabel("session count")
plt.legend(["Monkey 1", "Monkey 2"], frameon=False)
# plt.savefig("session_num_hist_filter.pdf")

### Row 1 B

In [ ]:
for monkey in ["1", "2"]:
    plot_data = loadmat("monkey" + monkey + "_filter.mat")["data"][0]
    pd.Series([i[0][0][0].shape[0] for i in plot_data]).hist(
        grid=False, bins=range(0, 2000, 100), alpha=0.7
    )
    print(
        np.mean([filter_data_sample(i)[0].shape[0] for i in get_data(monkey)]),
        np.std([filter_data_sample(i)[0].shape[0] for i in get_data(monkey)]),
    )
plt.xlabel("effective neuron count")
plt.ylabel("session count")
plt.legend(["Monkey 1", "Monkey 2"], frameon=False)
# plt.savefig("effective_neuron_cnt_filter.pdf")

### Row 2 A

In [ ]:
bz = 3
for monkey_num in ["1", "2"]:
    data = get_data(monkey_num)
    effective_list = []
    for s in data:
        data_sample = filter_data_sample(s)  # select session one
        dfs = split_data(
            data_sample, bin_size=bz
        )  # split stimulus data (r) based on bin size
        key_list = list(dfs.keys())

        pvalues = f_oneway(*[i for i in list(dfs.values()) if i.shape[0] > 0])[1]
        effective_neurons = sum(pvalues < 0.05)
        effective_list.append(effective_neurons)
    #     print(np.mean(effective_list), np.std(effective_list))
    pd.Series(effective_list).hist(grid=False, bins=range(0, 100, 5), alpha=0.7)
plt.legend(["Monkey 1", "Monkey 2"], frameon=False)
plt.xlabel("effective neuron count")
plt.ylabel("session count")
# plt.savefig("effective_neuron_cnt.pdf")

### Row 2 B

In [ ]:
bz = 3
for monkey_num in ["1", "2"]:
    #     data = get_data(monkey_num)
    plot_data = loadmat("monkey" + monkey_num + "_filter.mat")["data"][0]
    data = [[i[0][0][0], i[0][0][4]] for i in plot_data]
    effective_list = []
    for s in data:
        data_sample = filter_data_sample(s)  # select session one
        if data_sample[0].shape[0] == 0:
            continue
        dfs = split_data(
            data_sample, bin_size=bz
        )  # split stimulus data (r) based on bin size
        key_list = list(dfs.keys())

        pvalues = f_oneway(*[i for i in list(dfs.values()) if i.shape[0] > 0])[1]
        effective_neurons = sum(pvalues < 0.05)
        effective_list.append(effective_neurons)
    #     print(np.mean(effective_list), np.std(effective_list))
    pd.Series(effective_list).hist(grid=False, bins=range(0, 100, 5), alpha=0.7)
plt.legend(["Monkey 1", "Monkey 2"], frameon=False)
plt.xlabel("effective neuron count")
plt.ylabel("session count")
# plt.savefig("effective_neuron_cnt.pdf")